In [ ]:
import h5py
import zarr
import numpy as np
import time
import os
import glob
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
flist = glob.glob("/data/data2/south-data-ejm/hdd/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/*2021-11-02*")
flist = sorted(flist)

In [ ]:
bucket = f"s3://Zarr-OOI-DAS-{chunk_size}/"
z = zarr.open_group(f"{bucket}", 
               storage_options = {
                   # for public data (anonymous access):
                   # "anon": True,
                    "key": "JFo3e3DJY2tgEMp9",
                    "secret": "MBCJB0kLAC8SqD62YjVqDtkErcDxyadu",
                    "client_kwargs": {
                       "endpoint_url": "http://pnwstore1.ess.washington.edu:9000"
                   }
               }, mode = 'a')

In [ ]:
# chunck size to alter
zrawdata = z.create_dataset("RawData", shape=(47500, 200*60*60*24*7), chunks=(chunk_size, 12000), dtype='i4')
zrawdata.attrs['_ARRAY_DIMENSIONS'] = ['channel', 'time']
zrawdata.attrs['overview.location'] = "offshore oregon"
zrawdata.attrs['overview.deployment_type'] = 'temporary'
zrawdata.attrs['overview.network'] = "OO"
zrawdata.attrs['overview.site_name'] = "regional cabled array"
zrawdata.attrs['overview.number_of_interrogators'] = 1
zrawdata.attrs['overview.principle_investigators'] = "University of Washington"
zrawdata.attrs['overview.start_datetime'] = "2021-11-01"
zrawdata.attrs['overview.end_datetime'] = "2021-11-07"
zrawdata.attrs['overview.purpose_of_collection'] = "experiment"
zrawdata.attrs['overview.collection_mode'] = "continuous"
zrawdata.attrs['overview.comment'] = "N/A"

zrawdata.attrs['cable_and_fiber.cable_fiber_id'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_start_time'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_end_time'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_characteristics'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_environment'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_model'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_diameter'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_coordinates'] = "N/A"
zrawdata.attrs['cable_and_fiber.cable_connector_coordinates'] = "N/A"
zrawdata.attrs['cable_and_fiber.fiber_mode'] = "continuous"
zrawdata.attrs['cable_and_fiber.fiber_refraction_index'] = "N/A"
zrawdata.attrs['cable_and_fiber.attenuation'] = "N/A"
zrawdata.attrs['cable_and_fiber.fiber_geometry'] = "N/A"
zrawdata.attrs['cable_and_fiber.winding_angle'] = "N/A"
zrawdata.attrs['cable_and_fiber.fiber_start_location'] = "N/A"
zrawdata.attrs['cable_and_fiber.fiber_end_location'] = "N/A"
zrawdata.attrs['cable_and_fiber.fiber_length'] = "N/A"
zrawdata.attrs['cable_and_fiber.comment'] = "N/A"

zrawdata.attrs['interrogator.interrogator_id'] = "N/A"
zrawdata.attrs['interrogator.manufacturer'] = "N/A"
zrawdata.attrs['interrogator.model'] = "N/A"
zrawdata.attrs['interrogator.unit_of_measure'] = "N/A"
zrawdata.attrs['interrogator.comment'] = "N/A"

zrawdata.attrs['acquisition.acquisition_id'] = "N/A"
zrawdata.attrs['acquisition.acquisition_start_time'] = "2021-11-02T00:00:14.000000"
zrawdata.attrs['acquisition.acquisition_end_time'] = "2021-11-02T00:09:14.000000"
zrawdata.attrs['acquisition.acquisition_sample_rate'] = 200
zrawdata.attrs['acquisition.pulse_repetition_rate'] = "N/A"
zrawdata.attrs['acquisition.interrogator_rate'] = "N/A"
zrawdata.attrs['acquisition.pulse_width'] = "N/A"
zrawdata.attrs['acquisition.gauge_length'] = "N/A"
zrawdata.attrs['acquisition.number_of_channels'] = "N/A"
zrawdata.attrs['acquisition.channel_spacing'] = "N/A"
zrawdata.attrs['acquisition.archived_sample_rate'] = "N/A"
zrawdata.attrs['acquisition.unit_of_measure'] = "N/A"
zrawdata.attrs['acquisition.decimation'] = "N/A"
zrawdata.attrs['acquisition.filtering'] = "N/A"
zrawdata.attrs['acquisition.comment'] = "N/A"

In [ ]:
t0 = time.time()
# sequentially convert 10 files
for idf, f in enumerate(flist[:10]):
    zname = f.split("_")[-1][:-3]
    print(f"{idf+1}: working on {zname}")
    f = h5py.File(f,'r')

    zrawdata[:, idf * 12000 : (idf+1) * 12000] = f['/Acquisition/Raw[0]/RawData'][:, :]

    f.close()
t1 = time.time()
zarr.consolidate_metadata(z.store)
print(chunk_size, (time.time() - t1), (t1 - t0))